In [1]:
### I WILL PUT THE SOLVER CODE HERE AND WE CAN MERGE TO THE MAIN FILE ONCE ITS WORKING
#IMPORT DEPENDENCIES
import pandas as pd
import yfinance as yf
import datetime
import numpy as np

In [2]:
#APIs
# Define the list of stock ticker symbols
tickers = ['AGG', 'BNDX', 'SPY', 'IVW', 'IVE', 'BTC-USD', 'ETH-USD']

# Calculate the start date (three years ago from today)
start_date = datetime.datetime.now() - datetime.timedelta(days=1095)

# Initialize an empty dataframe to store the Close columns for each ticker
new_df = pd.DataFrame()

# Initialize an empty list to store the column names of Close prices for each ticker
ticker_closes = []

In [3]:
#Past 3 years of data into data frames
# Iterate over each ticker symbol
for ticker in tickers:
    # Fetch historical stock price data for the current ticker
    stock_data = yf.download(ticker, start=start_date, end=datetime.datetime.now())
    
    # Extract the 'Close' column and rename it for the current ticker
    close_column = stock_data[['Close']].rename(columns={'Close': f'{ticker}_Close'})
    
    # Compute the daily returns and insert them to the left of the corresponding ticker's Close column
    daily_returns = close_column.pct_change().rename(columns={f'{ticker}_Close': f'{ticker}_Daily_Return'})
    new_df = pd.concat([daily_returns, new_df], axis=1)
    
    # Append the Close column name for the current ticker to the ticker_closes list
    ticker_closes.append(f'{ticker}_Close')

# Drop rows containing NaNs from the new_df dataframe
new_df.dropna(inplace=True)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
## Feel free to adjust or send feedback if the code could be better or be more precise to what is needed:

span = len(new_df)  # Adjust the span for three years of historical data

# Calculate the annualized exponentially weighted mean return for each asset
annualized_mean_return = new_df.ewm(span=span).mean().iloc[-1] * 252

# Calculate the annualized exponentially weighted standard deviation (risk) for each asset
annualized_std_dev = new_df.ewm(span=span).std().iloc[-1] * np.sqrt(252)

# Display annualized risk and return for each asset
for ticker in tickers:
    print(f'{ticker}:')
    print(f'Annualized Mean Return: {annualized_mean_return[f"{ticker}_Daily_Return"]:.2%}')
    print(f'Annualized Risk (Standard Deviation): {annualized_std_dev[f"{ticker}_Daily_Return"]:.2%}')
    print()

print(new_df)

AGG:
Annualized Mean Return: -3.87%
Annualized Risk (Standard Deviation): 7.05%

BNDX:
Annualized Mean Return: -2.80%
Annualized Risk (Standard Deviation): 6.06%

SPY:
Annualized Mean Return: 13.34%
Annualized Risk (Standard Deviation): 16.07%

IVW:
Annualized Mean Return: 14.92%
Annualized Risk (Standard Deviation): 19.57%

IVE:
Annualized Mean Return: 11.48%
Annualized Risk (Standard Deviation): 14.21%

BTC-USD:
Annualized Mean Return: 34.27%
Annualized Risk (Standard Deviation): 51.61%

ETH-USD:
Annualized Mean Return: 16.06%
Annualized Risk (Standard Deviation): 61.16%

            ETH-USD_Daily_Return  BTC-USD_Daily_Return  IVE_Daily_Return  \
Date                                                                       
2021-05-11              0.054755              0.015123         -0.013545   
2021-05-12             -0.091840             -0.133217         -0.017366   
2021-05-13             -0.018675              0.011509          0.013631   
2021-05-14              0.097953       

In [5]:
# Create a dictionary with calculated values
assets_data = {
    'Ticker': tickers,
    'Annualized_Mean_Return': [annualized_mean_return[f"{ticker}_Daily_Return"] for ticker in tickers],
    'Annualized_Std_Dev': [annualized_std_dev[f"{ticker}_Daily_Return"] for ticker in tickers]
}

# Convert the dictionary into a DataFrame
individual_assets = pd.DataFrame(assets_data)

# Print the DataFrame
print(individual_assets)

    Ticker  Annualized_Mean_Return  Annualized_Std_Dev
0      AGG               -0.038736            0.070512
1     BNDX               -0.027964            0.060610
2      SPY                0.133433            0.160694
3      IVW                0.149237            0.195662
4      IVE                0.114759            0.142062
5  BTC-USD                0.342737            0.516110
6  ETH-USD                0.160597            0.611579


In [6]:
from scipy.optimize import minimize
import numpy as np

agg_mean_return = individual_assets['Annualized_Mean_Return']

# Define the objective function to minimize
def objective_function(weights, mean_returns, target_return):
    # Calculate the portfolio return
    portfolio_return = np.dot(mean_returns, weights)
    # Calculate the squared difference between the portfolio return and the target return
    return (portfolio_return - target_return) ** 2

# Define initial guess for weights
initial_guess = np.ones(len(agg_mean_return)) / len(agg_mean_return)  # Equal weights initially

# Define the target return
target_return = 0.05  # Example target return (replace with your desired target return)

# Define constraint function to ensure weights sum to 1
def constraint(weights):
    return np.sum(weights) - 1

# Perform optimization
result = minimize(objective_function, initial_guess, args=(agg_mean_return, target_return), method='SLSQP', bounds=[(0, 1)] * len(agg_mean_return), constraints={'type': 'eq', 'fun': constraint})

# Print optimization result
print("Optimization Result:")
print(result)
print("Optimal solution:", result.x)

# Round the optimal solution to a certain number of decimal places
rounded_solution = np.round(result.x, decimals=4)

# Ensure that the rounded weights sum up to 1
rounded_solution /= np.sum(rounded_solution)

# Print the rounded and normalized solution
print("Optimal solution (rounded and normalized):", rounded_solution)

Optimization Result:
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 7.024864674728787e-19
       x: [ 2.603e-01  2.519e-01  1.267e-01  1.144e-01  1.412e-01
            2.168e-19  1.056e-01]
     nit: 6
     jac: [ 8.729e-11  5.853e-11  4.163e-11  8.171e-11  3.873e-12
            1.176e-09  1.151e-10]
    nfev: 48
    njev: 6
Optimal solution: [2.60285272e-01 2.51924891e-01 1.26660957e-01 1.14395587e-01
 1.41154583e-01 2.16840434e-19 1.05578710e-01]
Optimal solution (rounded and normalized): [0.26027397 0.25187481 0.12668733 0.11438856 0.14118588 0.
 0.10558944]


In [7]:
five_perc_return = pd.DataFrame(rounded_solution)
five_perc_return

,0
0,0.260274
1,0.251875
2,0.126687
3,0.114389
4,0.141186
5,0.000000
6,0.105589


In [8]:
import pandas as pd

# Assuming individual_assets is your DataFrame
individual_assets['five_perc_return_weights'] = five_perc_return

# Now individual_assets will have a new column 'test_weights'
print(individual_assets)


    Ticker  Annualized_Mean_Return  Annualized_Std_Dev  \
0      AGG               -0.038736            0.070512   
1     BNDX               -0.027964            0.060610   
2      SPY                0.133433            0.160694   
3      IVW                0.149237            0.195662   
4      IVE                0.114759            0.142062   
5  BTC-USD                0.342737            0.516110   
6  ETH-USD                0.160597            0.611579   

   five_perc_return_weights  
0                  0.260274  
1                  0.251875  
2                  0.126687  
3                  0.114389  
4                  0.141186  
5                  0.000000  
6                  0.105589  


In [9]:
import numpy as np

# Calculate the portfolio return using sum product
portfolio_return_fice_perc = np.dot(individual_assets['five_perc_return_weights'], individual_assets['Annualized_Mean_Return'])

print("Portfolio Return:", portfolio_return_fice_perc)


Portfolio Return: 0.050009622952000066


In [10]:
from scipy.optimize import minimize
import numpy as np

agg_mean_return = individual_assets['Annualized_Mean_Return']

# Define the objective function to minimize
def objective_function(weights, mean_returns, target_return):
    # Calculate the portfolio return
    portfolio_return = np.dot(mean_returns, weights)
    # Calculate the squared difference between the portfolio return and the target return
    return (portfolio_return - target_return) ** 2

# Define initial guess for weights
initial_guess = np.ones(len(agg_mean_return)) / len(agg_mean_return)  # Equal weights initially

# Define the target return
target_return = 0.10  # Example target return (replace with your desired target return)

# Define constraint function to ensure weights sum to 1
def constraint(weights):
    return np.sum(weights) - 1

# Perform optimization
result = minimize(objective_function, initial_guess, args=(agg_mean_return, target_return), method='SLSQP', bounds=[(0, 1)] * len(agg_mean_return), constraints={'type': 'eq', 'fun': constraint})

# Print optimization result
print("Optimization Result:")
print(result)
print("Optimal solution:", result.x)

# Round the optimal solution to a certain number of decimal places
rounded_solution = np.round(result.x, decimals=4)

# Ensure that the rounded weights sum up to 1
rounded_solution /= np.sum(rounded_solution)

# Print the rounded and normalized solution
print("Optimal solution (rounded and normalized):", rounded_solution)

Optimization Result:
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 8.255983672437994e-09
       x: [ 1.731e-01  1.711e-01  1.401e-01  1.371e-01  1.437e-01
            9.999e-02  1.349e-01]
     nit: 3
     jac: [-7.039e-06 -5.082e-06  2.425e-05  2.712e-05  2.085e-05
            6.229e-05  2.918e-05]
    nfev: 24
    njev: 3
Optimal solution: [0.17313136 0.17106589 0.14011873 0.1370885  0.14369946 0.09998582
 0.13491024]
Optimal solution (rounded and normalized): [0.1731 0.1711 0.1401 0.1371 0.1437 0.1    0.1349]


In [11]:
ten_perc_return = pd.DataFrame(rounded_solution)
ten_perc_return

,0
0,0.1731
1,0.1711
2,0.1401
3,0.1371
4,0.1437
5,0.1000
6,0.1349


In [12]:
import pandas as pd

# Assuming individual_assets is your DataFrame
individual_assets['ten_perc_return_weights'] = ten_perc_return

# Now individual_assets will have a new column 'test_weights'
print(individual_assets)

    Ticker  Annualized_Mean_Return  Annualized_Std_Dev  \
0      AGG               -0.038736            0.070512   
1     BNDX               -0.027964            0.060610   
2      SPY                0.133433            0.160694   
3      IVW                0.149237            0.195662   
4      IVE                0.114759            0.142062   
5  BTC-USD                0.342737            0.516110   
6  ETH-USD                0.160597            0.611579   

   five_perc_return_weights  ten_perc_return_weights  
0                  0.260274                   0.1731  
1                  0.251875                   0.1711  
2                  0.126687                   0.1401  
3                  0.114389                   0.1371  
4                  0.141186                   0.1437  
5                  0.000000                   0.1000  
6                  0.105589                   0.1349  


In [13]:
from scipy.optimize import minimize
import numpy as np

agg_mean_return = individual_assets['Annualized_Mean_Return']

# Define the objective function to minimize
def objective_function(weights, mean_returns, target_return):
    # Calculate the portfolio return
    portfolio_return = np.dot(mean_returns, weights)
    # Calculate the squared difference between the portfolio return and the target return
    return (portfolio_return - target_return) ** 2

# Define initial guess for weights
initial_guess = np.ones(len(agg_mean_return)) / len(agg_mean_return)  # Equal weights initially

# Define the target return
target_return = 0.15  # Example target return (replace with your desired target return)

# Define constraint function to ensure weights sum to 1
def constraint(weights):
    return np.sum(weights) - 1

# Perform optimization
result = minimize(objective_function, initial_guess, args=(agg_mean_return, target_return), method='SLSQP', bounds=[(0, 1)] * len(agg_mean_return), constraints={'type': 'eq', 'fun': constraint})

# Print optimization result
print("Optimization Result:")
print(result)
print("Optimal solution:", result.x)

# Round the optimal solution to a certain number of decimal places
rounded_solution = np.round(result.x, decimals=4)

# Ensure that the rounded weights sum up to 1
rounded_solution /= np.sum(rounded_solution)

# Print the rounded and normalized solution
print("Optimal solution (rounded and normalized):", rounded_solution)

Optimization Result:
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 2.1420943103854063e-08
       x: [ 9.409e-02  9.742e-02  1.473e-01  1.521e-01  1.415e-01
            2.119e-01  1.557e-01]
     nit: 3
     jac: [ 1.134e-05  8.186e-06 -3.906e-05 -4.368e-05 -3.359e-05
           -1.003e-04 -4.701e-05]
    nfev: 24
    njev: 3
Optimal solution: [0.09409399 0.09742089 0.14726795 0.15214879 0.14150041 0.21191063
 0.15565734]
Optimal solution (rounded and normalized): [0.0941 0.0974 0.1473 0.1521 0.1415 0.2119 0.1557]


In [14]:
fifteen_perc_return = pd.DataFrame(rounded_solution)
fifteen_perc_return

# Assuming individual_assets is your DataFrame
individual_assets['fifteen_perc_return_weights'] = fifteen_perc_return

# Now individual_assets will have a new column 'test_weights'
print(individual_assets)

    Ticker  Annualized_Mean_Return  Annualized_Std_Dev  \
0      AGG               -0.038736            0.070512   
1     BNDX               -0.027964            0.060610   
2      SPY                0.133433            0.160694   
3      IVW                0.149237            0.195662   
4      IVE                0.114759            0.142062   
5  BTC-USD                0.342737            0.516110   
6  ETH-USD                0.160597            0.611579   

   five_perc_return_weights  ten_perc_return_weights  \
0                  0.260274                   0.1731   
1                  0.251875                   0.1711   
2                  0.126687                   0.1401   
3                  0.114389                   0.1371   
4                  0.141186                   0.1437   
5                  0.000000                   0.1000   
6                  0.105589                   0.1349   

   fifteen_perc_return_weights  
0                       0.0941  
1                   

In [15]:
from scipy.optimize import minimize
import numpy as np

agg_mean_return = individual_assets['Annualized_Mean_Return']

# Define the objective function to minimize
def objective_function(weights, mean_returns, target_return):
    # Calculate the portfolio return
    portfolio_return = np.dot(mean_returns, weights)
    # Calculate the squared difference between the portfolio return and the target return
    return (portfolio_return - target_return) ** 2

# Define initial guess for weights
initial_guess = np.ones(len(agg_mean_return)) / len(agg_mean_return)  # Equal weights initially

# Define the target return
target_return = 0.20  # Example target return (replace with your desired target return)

# Define constraint function to ensure weights sum to 1
def constraint(weights):
    return np.sum(weights) - 1

# Perform optimization
result = minimize(objective_function, initial_guess, args=(agg_mean_return, target_return), method='SLSQP', bounds=[(0, 1)] * len(agg_mean_return), constraints={'type': 'eq', 'fun': constraint})

# Print optimization result
print("Optimization Result:")
print(result)
print("Optimal solution:", result.x)

# Round the optimal solution to a certain number of decimal places
rounded_solution = np.round(result.x, decimals=4)

# Ensure that the rounded weights sum up to 1
rounded_solution /= np.sum(rounded_solution)

# Print the rounded and normalized solution
print("Optimal solution (rounded and normalized):", rounded_solution)

Optimization Result:
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1.4713397171806511e-07
       x: [ 1.506e-02  2.378e-02  1.544e-01  1.672e-01  1.393e-01
            3.238e-01  1.764e-01]
     nit: 3
     jac: [ 2.972e-05  2.145e-05 -1.024e-04 -1.145e-04 -8.804e-05
           -2.629e-04 -1.232e-04]
    nfev: 24
    njev: 3
Optimal solution: [0.01505661 0.02377589 0.15441718 0.16720908 0.13930137 0.32383544
 0.17640445]
Optimal solution (rounded and normalized): [0.0151 0.0238 0.1544 0.1672 0.1393 0.3238 0.1764]


In [16]:
twenty_perc_return = pd.DataFrame(rounded_solution)
twenty_perc_return

# Assuming individual_assets is your DataFrame
individual_assets['twenty_perc_return_weights'] = twenty_perc_return

# Now individual_assets will have a new column 'test_weights'
print(individual_assets)

    Ticker  Annualized_Mean_Return  Annualized_Std_Dev  \
0      AGG               -0.038736            0.070512   
1     BNDX               -0.027964            0.060610   
2      SPY                0.133433            0.160694   
3      IVW                0.149237            0.195662   
4      IVE                0.114759            0.142062   
5  BTC-USD                0.342737            0.516110   
6  ETH-USD                0.160597            0.611579   

   five_perc_return_weights  ten_perc_return_weights  \
0                  0.260274                   0.1731   
1                  0.251875                   0.1711   
2                  0.126687                   0.1401   
3                  0.114389                   0.1371   
4                  0.141186                   0.1437   
5                  0.000000                   0.1000   
6                  0.105589                   0.1349   

   fifteen_perc_return_weights  twenty_perc_return_weights  
0                       0

In [17]:
from scipy.optimize import minimize
import numpy as np

agg_mean_return = individual_assets['Annualized_Mean_Return']

# Define the objective function to minimize
def objective_function(weights, mean_returns, target_return):
    # Calculate the portfolio return
    portfolio_return = np.dot(mean_returns, weights)
    # Calculate the squared difference between the portfolio return and the target return
    return (portfolio_return - target_return) ** 2

# Define initial guess for weights
initial_guess = np.ones(len(agg_mean_return)) / len(agg_mean_return)  # Equal weights initially

# Define the target return
target_return = 0.25  # Example target return (replace with your desired target return)

# Define constraint function to ensure weights sum to 1
def constraint(weights):
    return np.sum(weights) - 1

# Perform optimization
result = minimize(objective_function, initial_guess, args=(agg_mean_return, target_return), method='SLSQP', bounds=[(0, 1)] * len(agg_mean_return), constraints={'type': 'eq', 'fun': constraint})

# Print optimization result
print("Optimization Result:")
print(result)
print("Optimal solution:", result.x)

# Round the optimal solution to a certain number of decimal places
rounded_solution = np.round(result.x, decimals=4)

# Ensure that the rounded weights sum up to 1
rounded_solution /= np.sum(rounded_solution)

# Print the rounded and normalized solution
print("Optimal solution (rounded and normalized):", rounded_solution)

Optimization Result:
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1.2456151405017568e-17
       x: [ 3.934e-18  9.525e-19  1.047e-01  1.369e-01  6.657e-02
            5.318e-01  1.601e-01]
     nit: 6
     jac: [ 2.958e-10  2.090e-10 -6.766e-10 -7.215e-10 -6.138e-10
           -6.688e-10 -7.493e-10]
    nfev: 48
    njev: 6
Optimal solution: [3.93398307e-18 9.52496096e-19 1.04670206e-01 1.36916840e-01
 6.65653096e-02 5.31750532e-01 1.60097112e-01]
Optimal solution (rounded and normalized): [0.         0.         0.10468953 0.13688631 0.06659334 0.53174683
 0.16008399]


In [18]:
twentyfive_perc_return = pd.DataFrame(rounded_solution)
twentyfive_perc_return

# Assuming individual_assets is your DataFrame
individual_assets['twentyfive_perc_return_weights'] = twentyfive_perc_return

# Now individual_assets will have a new column 'test_weights'
print(individual_assets)

    Ticker  Annualized_Mean_Return  Annualized_Std_Dev  \
0      AGG               -0.038736            0.070512   
1     BNDX               -0.027964            0.060610   
2      SPY                0.133433            0.160694   
3      IVW                0.149237            0.195662   
4      IVE                0.114759            0.142062   
5  BTC-USD                0.342737            0.516110   
6  ETH-USD                0.160597            0.611579   

   five_perc_return_weights  ten_perc_return_weights  \
0                  0.260274                   0.1731   
1                  0.251875                   0.1711   
2                  0.126687                   0.1401   
3                  0.114389                   0.1371   
4                  0.141186                   0.1437   
5                  0.000000                   0.1000   
6                  0.105589                   0.1349   

   fifteen_perc_return_weights  twenty_perc_return_weights  \
0                       

In [19]:
from scipy.optimize import minimize
import numpy as np

agg_mean_return = individual_assets['Annualized_Mean_Return']

# Define the objective function to minimize
def objective_function(weights, mean_returns, target_return):
    # Calculate the portfolio return
    portfolio_return = np.dot(mean_returns, weights)
    # Calculate the squared difference between the portfolio return and the target return
    return (portfolio_return - target_return) ** 2

# Define initial guess for weights
initial_guess = np.ones(len(agg_mean_return)) / len(agg_mean_return)  # Equal weights initially

# Define the target return
target_return = 0.30  # Example target return (replace with your desired target return)

# Define constraint function to ensure weights sum to 1
def constraint(weights):
    return np.sum(weights) - 1

# Perform optimization
result = minimize(objective_function, initial_guess, args=(agg_mean_return, target_return), method='SLSQP', bounds=[(0, 1)] * len(agg_mean_return), constraints={'type': 'eq', 'fun': constraint})

# Print optimization result
print("Optimization Result:")
print(result)
print("Optimal solution:", result.x)

# Round the optimal solution to a certain number of decimal places
rounded_solution = np.round(result.x, decimals=4)

# Ensure that the rounded weights sum up to 1
rounded_solution /= np.sum(rounded_solution)

# Print the rounded and normalized solution
print("Optimal solution (rounded and normalized):", rounded_solution)

Optimization Result:
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1.2913298086786737e-17
       x: [ 0.000e+00  0.000e+00  2.402e-02  8.065e-02  0.000e+00
            7.740e-01  1.214e-01]
     nit: 8
     jac: [ 3.008e-10  2.126e-10 -6.937e-10 -7.407e-10 -6.285e-10
           -7.128e-10 -7.699e-10]
    nfev: 64
    njev: 8
Optimal solution: [0.         0.         0.02402306 0.08064814 0.         0.7739761
 0.1213527 ]
Optimal solution (rounded and normalized): [0.     0.     0.024  0.0806 0.     0.774  0.1214]


In [20]:
thirty_perc_return = pd.DataFrame(rounded_solution)
thirty_perc_return

# Assuming individual_assets is your DataFrame
individual_assets['thirty_perc_return_weights'] = thirty_perc_return

# Now individual_assets will have a new column 'test_weights'
print(individual_assets)

    Ticker  Annualized_Mean_Return  Annualized_Std_Dev  \
0      AGG               -0.038736            0.070512   
1     BNDX               -0.027964            0.060610   
2      SPY                0.133433            0.160694   
3      IVW                0.149237            0.195662   
4      IVE                0.114759            0.142062   
5  BTC-USD                0.342737            0.516110   
6  ETH-USD                0.160597            0.611579   

   five_perc_return_weights  ten_perc_return_weights  \
0                  0.260274                   0.1731   
1                  0.251875                   0.1711   
2                  0.126687                   0.1401   
3                  0.114389                   0.1371   
4                  0.141186                   0.1437   
5                  0.000000                   0.1000   
6                  0.105589                   0.1349   

   fifteen_perc_return_weights  twenty_perc_return_weights  \
0                       

In [21]:
from scipy.optimize import minimize
import numpy as np

agg_mean_return = individual_assets['Annualized_Mean_Return']

# Define the objective function to minimize
def objective_function(weights, mean_returns, target_return):
    # Calculate the portfolio return
    portfolio_return = np.dot(mean_returns, weights)
    # Calculate the squared difference between the portfolio return and the target return
    return (portfolio_return - target_return) ** 2

# Define initial guess for weights
initial_guess = np.ones(len(agg_mean_return)) / len(agg_mean_return)  # Equal weights initially

# Define the target return
target_return = 0.35  # Example target return (replace with your desired target return)

# Define constraint function to ensure weights sum to 1
def constraint(weights):
    return np.sum(weights) - 1

# Perform optimization
result = minimize(objective_function, initial_guess, args=(agg_mean_return, target_return), method='SLSQP', bounds=[(0, 1)] * len(agg_mean_return), constraints={'type': 'eq', 'fun': constraint})

# Print optimization result
print("Optimization Result:")
print(result)
print("Optimal solution:", result.x)

# Round the optimal solution to a certain number of decimal places
rounded_solution = np.round(result.x, decimals=4)

# Ensure that the rounded weights sum up to 1
rounded_solution /= np.sum(rounded_solution)

# Print the rounded and normalized solution
print("Optimal solution (rounded and normalized):", rounded_solution)

Optimization Result:
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 5.275628186880968e-05
       x: [ 3.022e-18  3.117e-18  0.000e+00  0.000e+00  0.000e+00
            1.000e+00  2.776e-17]
     nit: 9
     jac: [ 5.627e-04  4.062e-04 -1.938e-03 -2.168e-03 -1.667e-03
           -4.979e-03 -2.333e-03]
    nfev: 72
    njev: 9
Optimal solution: [3.02204708e-18 3.11687288e-18 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 2.77555756e-17]
Optimal solution (rounded and normalized): [0. 0. 0. 0. 0. 1. 0.]


In [22]:
thirtyfive_perc_return = pd.DataFrame(rounded_solution)
thirtyfive_perc_return

# Assuming individual_assets is your DataFrame
individual_assets['thirtyfive_perc_return_weights'] = thirtyfive_perc_return

# Now individual_assets will have a new column 'test_weights'
print(individual_assets)

    Ticker  Annualized_Mean_Return  Annualized_Std_Dev  \
0      AGG               -0.038736            0.070512   
1     BNDX               -0.027964            0.060610   
2      SPY                0.133433            0.160694   
3      IVW                0.149237            0.195662   
4      IVE                0.114759            0.142062   
5  BTC-USD                0.342737            0.516110   
6  ETH-USD                0.160597            0.611579   

   five_perc_return_weights  ten_perc_return_weights  \
0                  0.260274                   0.1731   
1                  0.251875                   0.1711   
2                  0.126687                   0.1401   
3                  0.114389                   0.1371   
4                  0.141186                   0.1437   
5                  0.000000                   0.1000   
6                  0.105589                   0.1349   

   fifteen_perc_return_weights  twenty_perc_return_weights  \
0                       

In [28]:
# Calculate the correlation matrix
correlation_matrix = new_df.corr()
print(correlation_matrix)

weights = individual_assets[['five_perc_return_weights', 
                             'ten_perc_return_weights', 
                             'fifteen_perc_return_weights', 
                             'twenty_perc_return_weights', 
                             'twentyfive_perc_return_weights', 
                             'thirty_perc_return_weights', 
                             'thirty_perc_return_weights']].values
print(weights.T.shape)
# Define a function to calculate portfolio standard deviation
def calculate_portfolio_std(weights, correlation_matrix, std_deviations):
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(correlation_matrix, weights)) * np.sum(std_deviations ** 2))
    return portfolio_std

# Create a new DataFrame to store portfolio standard deviations
port_stds = pd.DataFrame(index=[f"Portfolio {i}" for i in range(1, 8)], columns=['Portfolio_Std_Dev'])

# Iterate over each row in the DataFrame
for index, row in individual_assets.iterrows():
    # Extract weights and standard deviations from the current row
    weights = row[['five_perc_return_weights', 
                             'ten_perc_return_weights', 
                             'fifteen_perc_return_weights', 
                             'twenty_perc_return_weights', 
                             'twentyfive_perc_return_weights', 
                             'thirty_perc_return_weights', 
                             'thirty_perc_return_weights']].values
    std_deviations = row['Annualized_Std_Dev']

    portfolio_std = calculate_portfolio_std(weights, correlation_matrix, std_deviations)
    
    # Add portfolio standard deviation to the port_stds DataFrame
    port_stds.at[f"Portfolio {index + 1}", 'Portfolio_Std_Dev'] = portfolio_std

# Print the DataFrame with portfolio standard deviations
print(port_stds)

                      ETH-USD_Daily_Return  BTC-USD_Daily_Return  \
ETH-USD_Daily_Return              1.000000              0.856107   
BTC-USD_Daily_Return              0.856107              1.000000   
IVE_Daily_Return                  0.328425              0.328808   
IVW_Daily_Return                  0.410967              0.398503   
SPY_Daily_Return                  0.396634              0.389145   
BNDX_Daily_Return                 0.046394              0.068353   
AGG_Daily_Return                  0.092680              0.107057   

                      IVE_Daily_Return  IVW_Daily_Return  SPY_Daily_Return  \
ETH-USD_Daily_Return          0.328425          0.410967          0.396634   
BTC-USD_Daily_Return          0.328808          0.398503          0.389145   
IVE_Daily_Return              1.000000          0.798704          0.922912   
IVW_Daily_Return              0.798704          1.000000          0.965297   
SPY_Daily_Return              0.922912          0.965297         

In [46]:

# Calculate the portfolio return using sum product
portfolio_return_fice_perc = (np.dot(individual_assets['five_perc_return_weights'], individual_assets['Annualized_Mean_Return']))*100
portfolio_return_fice_sharpe = portfolio_return_fice_perc/(port_stds.iloc[0]['Portfolio_Std_Dev']*100)
print("Portfolio Return:", portfolio_return_fice_perc)
print("Portfolio Sharpe:", portfolio_return_fice_sharpe)

# Calculate the portfolio return using sum product
portfolio_return_ten_perc = (np.dot(individual_assets['ten_perc_return_weights'], individual_assets['Annualized_Mean_Return']))*100
portfolio_return_ten_sharpe = portfolio_return_ten_perc/(port_stds.iloc[1]['Portfolio_Std_Dev']*100)
print("Portfolio Return:", portfolio_return_ten_perc)
print("Portfolio Sharpe:", portfolio_return_ten_sharpe)

# Calculate the portfolio return using sum product
portfolio_return_fifteen_perc = (np.dot(individual_assets['fifteen_perc_return_weights'], individual_assets['Annualized_Mean_Return']))*100
portfolio_return_fifteen_sharpe = portfolio_return_fifteen_perc/(port_stds.iloc[2]['Portfolio_Std_Dev']*100)
print("Portfolio Return:", portfolio_return_fifteen_perc)
print("Portfolio Sharpe:", portfolio_return_fifteen_sharpe)

# Calculate the portfolio return using sum product
portfolio_return_twenty_perc = (np.dot(individual_assets['twenty_perc_return_weights'], individual_assets['Annualized_Mean_Return']))*100
portfolio_return_twenty_sharpe = portfolio_return_twenty_perc/(port_stds.iloc[3]['Portfolio_Std_Dev']*100)
print("Portfolio Return:", portfolio_return_twenty_perc)
print("Portfolio Sharpe:", portfolio_return_twenty_sharpe)

# Calculate the portfolio return using sum product
portfolio_return_twentyfive_perc = (np.dot(individual_assets['twentyfive_perc_return_weights'], individual_assets['Annualized_Mean_Return']))*100
portfolio_return_twentyfive_sharpe = portfolio_return_twentyfive_perc/(port_stds.iloc[4]['Portfolio_Std_Dev']*100)
print("Portfolio Return:", portfolio_return_twentyfive_perc)
print("Portfolio Sharpe:", portfolio_return_twentyfive_sharpe)

Portfolio Return: 5.000962295200006
Portfolio Sharpe: 1.5179249466360716
Portfolio Return: 10.009361714559349
Portfolio Sharpe: 3.5573890621738635
Portfolio Return: 14.985414667562406
Portfolio Sharpe: 1.6720223869029278
Portfolio Return: 19.959740045841027
Portfolio Sharpe: 1.6225950823387572
Portfolio Return: 24.999785830364115
Portfolio Sharpe: 3.4545909931407213


In [42]:
# TODO --> For Loop the above
for 
    portfolio_return_fice_perc = (np.dot(individual_assets['five_perc_return_weights'], individual_assets['Annualized_Mean_Return']))*100
    portfolio_return_fice_sharpe = portfolio_return_fice_perc/(port_stds.iloc[0]['Portfolio_Std_Dev']*100)
    print("Portfolio Return:", portfolio_return_fice_perc)
    print("Portfolio Sharpe:", portfolio_return_fice_sharpe)


Portfolio: five_perc_return_weights
Portfolio Return: 5.000962295200006
Portfolio Sharpe: 0.11863533518940125

Portfolio: ten_perc_return_weights
Portfolio Return: 10.009361714559349
Portfolio Sharpe: 0.2374470975672928

Portfolio: fifteen_perc_return_weights
Portfolio Return: 14.985414667562406
Portfolio Sharpe: 0.35549152085085567

Portfolio: twenty_perc_return_weights
Portfolio Return: 19.959740045841027
Portfolio Sharpe: 0.47349496174055106

Portfolio: twentyfive_perc_return_weights
Portfolio Return: 24.999785830364115
Portfolio Sharpe: 0.5930574550612314

Portfolio: thirty_perc_return_weights
Portfolio Return: 30.000552193549236
Portfolio Sharpe: 0.7116881422531298

Portfolio: thirtyfive_perc_return_weights
Portfolio Return: 34.27366480280239
Portfolio Sharpe: 0.8130570622282663

Portfolio: Portfolio_Std_Dev
Portfolio Return: 45.62767370878251
Portfolio Sharpe: 1.0824025547142335



In [38]:
for i in range(1,8):
    print(i)

1
2
3
4
5
6
7
